# `train_ratings.csv` 로드해서 `MostPopular`  추천 결과 생성하기

In [ ]:
# pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# import chardet

In [2]:
train_df = pd.read_csv('/data/ephemeral/home/movie/data/train/train_ratings.csv') # 전체 학습 데이터
titles_df = pd.read_csv('/data/ephemeral/home/movie/data/train/titles.tsv', sep='\t')
writers_df = pd.read_csv('/data/ephemeral/home/movie/data/train/writers.tsv', sep='\t')
years_df = pd.read_csv('/data/ephemeral/home/movie/data/train/years.tsv', sep='\t')
genres_df = pd.read_csv('/data/ephemeral/home/movie/data/train/genres.tsv', sep='\t')
directors_df = pd.read_csv('/data/ephemeral/home/movie/data/train/directors.tsv', sep='\t')
# Ml_item2attributes.json  directors.tsv  genres.tsv  titles.tsv  train_ratings.csv  writers.tsv  years.tsv

In [3]:
train_df.head()

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563


In [4]:
train_df.describe()

,user,item,time
count,5.154471e+06,5.154471e+06,5.154471e+06
mean,6.875153e+04,1.716849e+04,1.248017e+09
std,3.993460e+04,2.633563e+04,8.968302e+07
min,1.100000e+01,1.000000e+00,1.113221e+09
25%,3.419900e+04,1.304000e+03,1.170355e+09
50%,6.860500e+04,3.897000e+03,1.234651e+09
75%,1.032010e+05,2.671200e+04,1.318284e+09
max,1.384930e+05,1.191450e+05,1.427781e+09


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154471 entries, 0 to 5154470
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   user    int64
 1   item    int64
 2   time    int64
dtypes: int64(3)
memory usage: 118.0 MB


In [6]:
titles_df.head()

,item,title
0,318,"Shawshank Redemption, The (1994)"
1,2571,"Matrix, The (1999)"
2,2959,Fight Club (1999)
3,296,Pulp Fiction (1994)
4,356,Forrest Gump (1994)


In [7]:
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6807 entries, 0 to 6806
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   item    6807 non-null   int64 
 1   title   6807 non-null   object
dtypes: int64(1), object(1)
memory usage: 106.5+ KB


In [8]:
writers_df.head()

,item,writer
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005


In [9]:
writers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11306 entries, 0 to 11305
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   item    11306 non-null  int64 
 1   writer  11306 non-null  object
dtypes: int64(1), object(1)
memory usage: 176.8+ KB


In [10]:
years_df.head()

,item,year
0,1348,1922
1,44587,1922
2,4768,1922
3,8235,1923
4,8609,1923


In [11]:
years_df.nunique()

item    6799
year      93
dtype: int64

In [12]:
years_df.describe()

,item,year
count,6799.000000,6799.000000
mean,26630.017503,1992.174732
std,32187.872806,19.052568
min,1.000000,1922.000000
25%,3052.500000,1985.000000
50%,6879.000000,1999.000000
75%,49840.000000,2006.000000
max,119141.000000,2014.000000


In [13]:
years_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6799 entries, 0 to 6798
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   item    6799 non-null   int64
 1   year    6799 non-null   int64
dtypes: int64(2)
memory usage: 106.4 KB


In [14]:
years_df.isna().sum()

item    0
year    0
dtype: int64

In [15]:
genres_df.head()

,item,genre
0,318,Crime
1,318,Drama
2,2571,Action
3,2571,Sci-Fi
4,2571,Thriller


In [16]:
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15933 entries, 0 to 15932
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   item    15933 non-null  int64 
 1   genre   15933 non-null  object
dtypes: int64(1), object(1)
memory usage: 249.1+ KB


In [17]:
len(genres_df["genre"].unique())

18

In [18]:
directors_df.head()

,item,director
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005


In [19]:
directors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5905 entries, 0 to 5904
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item      5905 non-null   int64 
 1   director  5905 non-null   object
dtypes: int64(1), object(1)
memory usage: 92.4+ KB


In [20]:
directors_df["director"]

0       nm0000005
1       nm0000005
2       nm0000005
3       nm0000005
4       nm0000005
          ...    
5900    nm2879822
5901    nm2879822
5902    nm2879822
5903    nm9054338
5904    nm9054338
Name: director, Length: 5905, dtype: object

In [23]:
# _titles_df = pd.read_csv('/ephemeral/home/movie/data/train/._titles.tsv', sep='\t', encoding='ISO-8859-1')
# with open('../../../data/train/._titles.tsv', 'rb') as f:
#     result = chardet.detect(f.read())
#     print(result)
# _titles_df = pd.read_csv('../../../data/train/._titles.tsv', sep='\t', encoding='Windows-1252')
# _titles_df

In [25]:
len(directors_df["item"].unique())

5503

In [26]:
print(len(directors_df["item"].unique()))
print(len(genres_df["item"].unique()))
print(len(titles_df["item"].unique()))
print(len(writers_df["item"].unique()))
print(len(years_df["item"].unique()))
print(len(train_df["item"].unique()))

5503
6807
6807
5648
6799
6807


In [27]:
print(len(titles_df["title"].unique()))
print(len(titles_df["item"].unique()))

6806
6807


In [28]:
print(len(directors_df["item"].unique()))
print(len(directors_df["director"].unique()))
print(len(genres_df["item"].unique()))
print(len(genres_df["genre"].unique()))


5503
1340
6807
18


In [29]:
item_director_counts = directors_df.groupby('item')['director'].nunique().reset_index()
item_director_counts.columns = ['item', 'director_count']

director_counts = item_director_counts.groupby('director_count').count().reset_index()
director_counts.columns = ['director_count', 'count']
director_counts["count"].sum()

5503

In [30]:
item_genre_counts = genres_df.groupby('item')['genre'].nunique().reset_index()
item_genre_counts.columns = ['item', 'genre_count']

genre_counts = item_genre_counts.groupby('genre_count').count().reset_index()
genre_counts.columns = ['genre_count', 'count']
genre_counts["count"].sum()

6807

In [31]:
titles_df

,item,title
0,318,"Shawshank Redemption, The (1994)"
1,2571,"Matrix, The (1999)"
2,2959,Fight Club (1999)
3,296,Pulp Fiction (1994)
4,356,Forrest Gump (1994)
...,...,...
6802,73106,American Pie Presents: The Book of Love (Ameri...
6803,109850,Need for Speed (2014)
6804,8605,Taxi 3 (2003)
6805,3689,Porky's II: The Next Day (1983)


In [32]:
print(len(titles_df["item"].unique()))
print(len(titles_df["title"].unique()))

6807
6806


## 모든 사용자에게 동일하게 Most Popular 아이템 10개를 추천 결과로 제공

In [ ]:
# # most popular 10 items
# num_users = train_df['user'].nunique()
# mp_items = train_df['item'].value_counts()[:10].index.values # predictions
# mp_items = np.concatenate([mp_items]*num_users) # same prediction for all users
# users = train_df['user'].unique().repeat(10)

# test_df = pd.DataFrame(zip(users,mp_items), columns=['user','item'])
# test_df.to_csv("output/most_popular_submission.csv", index=False)

In [ ]:
# test_df

,user,item
0,11,2571
1,11,2959
2,11,296
3,11,318
4,11,356
...,...,...
313595,138493,4993
313596,138493,7153
313597,138493,5952
313598,138493,593
